In [97]:
import pandas as pd
import pulp as lp
import numpy as np
from operator import iadd
from functools import reduce
from itertools import chain
from typing import Sequence, Any, DefaultDict, List

The below functions assist in creating an lp model problem using `pulp` package 

In [98]:
def create_prob(prob_name: str, sense: int) -> lp.LpProblem:
    return lp.LpProblem(prob_name, sense)


def add_obj_fn(lp_prob: lp.LpProblem, dvar: lp.LpAffineExpression) -> lp.LpProblem:
    return iadd(lp_prob, dvar)


def add_constraint(lp_prob: lp.LpProblem, constrs: Sequence[lp.LpConstraint]) -> lp.LpProblem:
   return reduce(iadd, constrs, lp_prob)


def head(x: Sequence) -> Any:
    return x[0]

def to_str(indnum, activity) -> str:
    return f'{indnum} - {activity}'

# Loading The Prepared Data

We load our dataset into `activity_df`. `activity_df` contains all the observations related to individuals, their activities, consumption per unit and the relavant sources activities use.

In [99]:
activity_df = pd.read_csv('data/dataset.csv')

activity_df.drop(['Unnamed: 0', 'Group'], axis=1, inplace=True)

We load the sources carbon footprint dataset into `source_df`. `source_df` contains all the observations related to an carbon footprint an activity contains given that they are used with a specific source.

In [100]:
source_df = pd.read_csv('data/dataset_source_cf.csv')

source_df.drop(['Unnamed: 0', 'X__1'], axis=1, inplace=True)

In [101]:
source_df.head()

,Activity,Per,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,jetfuel,waste management,hybrid,electric_peak_hours,electric_off_peak_hours
0,Household heating => 70F,hour,0.000000,0.000000,0.000000,0.000000,0.0,0.000436,0.0,0.0,0.0,0.000650,0.000542
1,Household heating < 70F,hour,0.000000,0.000000,0.000000,0.000000,0.0,0.000872,0.0,0.0,0.0,0.000923,0.000901
2,Use of heat pump,hour,0.000000,0.000000,0.000000,0.000000,0.0,0.001074,0.0,0.0,0.0,0.001229,0.001188
3,Use of air conditioner,hour,0.000000,0.000000,0.000000,0.000000,0.0,0.000598,0.0,0.0,0.0,0.007980,0.000721
4,shower - short,activity,0.000012,0.000102,0.000232,0.000199,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000


# Building The Model

`lp_prob` is the linear programming problem we have to formulize. The problem is one in which we have to minimize the objective function. 

In [102]:
lp_prob = create_prob('Wells Fargo Challenge', lp.LpMinimize)

## Defining The Decision Variables

The `individuals` variable represents all the indviduals that we want to focus on, here we are looking at the first __10__ individuals. `MAX_NUM_INDV` is used to change the maximum number of individuals the linear programming model should solve for.

The `activities` variable is the list of activities that all indiviuals do.

The `sources` variables is the list of all sources that an activity can use.

In [103]:
MAX_NUM_INDV = 2

In [104]:
individuals = activity_df.Indnum.unique()[0:MAX_NUM_INDV]
activities = activity_df.Activity.unique()

M = 100 # The big number M 

sources = [
  "solar_powered_water_heater",
  "gas_water_heater",
  "electric_water_heater_peak_hour",
  "electric_water_heater_off_peak",
  "gas",
  "natural_gas",
  "hybrid",
  "electric_peak_hours",
  "electric_off_peak_hours",
  "jetfuel",
  "waste management"
]

Our decision variable is called 

### $S_{ijk}$

where 
* i is individual
* j is activity
* k is source

It is binary variable that can have either 0 or 1. Meaning that $S_{ijk}$ represents wether an individual uses a particular source or not.

In [105]:
source_indexes = [
    (f'{indv} - {activity}', source)
    for indv in individuals
    for activity in activities
    for source in sources
]

S_ijk = lp.LpVariable.dicts('S_ijk', source_indexes, lowBound=0, cat='Binary')

## The Objective Function

### $Z_{min} = \sum S_{ijk} *  SCF_{ijk} * C_{ij}$

where
* i = 1...n
* j = 1...27
* k = 1...10

`C` is the consumption per unit of an activity while `SCF` is the carbon footprint per source.

$C_{ij}$ is assumed to be __constant__ for each individual and activity and $SCF_{ijk}$ is assumed to be __constant__ for per source.

In the case that $C_{ij}$ is 0 we will use the big `M` method to enforce a big penalty, this will ensure that the linear programming model won't choose that particular source

In [106]:
d_vars = []

for indv in individuals:
    for activity in activities:
        consumption: np.ndarray = activity_df.loc[activity_df['Activity'] == activity, 'Consumption'].values
            
        for source in sources:
            source_cf: np.ndarray = source_df.loc[source_df['Activity'] == activity, source].values
            source_cf = M if head(source_cf) == 0.0 else head(source_cf)
            d_vars.append(S_ijk[(to_str(indv, activity), source)] * float(source_cf) * head(consumption))


obj_fn = lp.lpSum(sum(d_vars))

## The Constraints

The constraint below ensures that an activity can only use one source. The goal of this constraint is to use a single source that has the lowest carbon footprint.

### $\sum_k^m S_{ijk} == 1$
where 
* n = m = 10
* i = 1...n
* j = 1...27

In [107]:
sum_source_conds = []

for indv in individuals:
    for activity in activities:
        sum_source = []
        for source in sources:
            sum_source.append(S_ijk[(to_str(indv, activity), source)])
            
        sum_source_conds.append(lp.lpSum(sum_source) == 1)

We add the objective function and the contraints to the model

In [108]:
lp_prob = add_obj_fn(lp_prob, obj_fn)
lp_prob = add_constraint(lp_prob, sum_source_conds)

In [109]:
lp_prob.solve()
lp.LpStatus[lp_prob.status]

'Optimal'

The minimum carbon footprint that can be obtained for 10 individuals is 

In [110]:
lp.value(lp_prob.objective)

3.238865999999999

In [111]:
columns = list(chain(['Individual'], ["Activity"], sources))

In [112]:
rows = []

for indv in individuals:
    for activity in activities:
        row = [indv, activity]
        
        row.extend([lp.value(S_ijk[(to_str(indv, activity), source)]) for source in sources])
        
        rows.append(row)
 
table_df = pd.DataFrame(rows, columns=columns) 

In [113]:
table_df

,Individual,Activity,solar_powered_water_heater,gas_water_heater,electric_water_heater_peak_hour,electric_water_heater_off_peak,gas,natural_gas,hybrid,electric_peak_hours,electric_off_peak_hours,jetfuel,waste management
0,1,Household heating => 70F,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,Household heating < 70F,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,Use of heat pump,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,Use of air conditioner,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,shower - short,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,shower - long (> 3 min),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,bath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1,wash-up,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,use of dishwasher,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1,use of clothes washer,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
